<a href="https://colab.research.google.com/github/nastya-biran/scheduler/blob/main/%D0%A1%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 50.3 MB/s eta 0:00:00


In [2]:
from pulp import *
import numpy as np
import pandas as pd
import re

In [3]:
teachers_data = pd.read_csv("Teachers.csv")
groups_data = pd.read_csv("Groups.csv")
limits_data = pd.read_csv("Teachers_limits.csv")

In [4]:
teachers_data.head(5)

,Преподаватель,"Зельеварение, семинар","Зельеварение, лекция","Защита от темных сил, лекция","Защита от темных сил, семинар","Трансфигурация, лекция","Трансфигурация, семинар",Понедельник,Вторник,Среда,Четверг,Пятница,Суббота
0,Северус Снейп,1,1,0,1,0,0,NaN,"16.20-17.40, 18.10-19.30",NaN,"9.30-10.50, 11.10-12.30",NaN,NaN
1,Минерва Макгоналл,0,0,1,1,0,1,"11.10-12.30, 13.00-14.20, 14.40-16.00, 16.20-1...",NaN,NaN,NaN,NaN,NaN
2,Альбус Дамблдор,1,0,0,0,1,1,"11.10-12.30, 13.00-14.20",NaN,NaN,"9.30-10.50, 11.10-12.30",NaN,NaN


In [5]:
groups_data.head(5)

,Номер группы,"Зельеварение, лекция","Зельеварение, семинар","Защита от темных сил, лекция","Защита от темных сил, семинар","Трансфигурация, лекция","Трансфигурация, семинар"
0,1,2,2,1,1,0,0
1,2,2,2,1,1,0,0
2,3,0,0,1,1,2,2
3,4,0,0,1,1,2,2


In [6]:
limits_data.head(5)

,Преподаватель,Все пары в один день,Дни без пар,Количество пар в неделю
0,Северус Снейп,NaN,"Понедельник, Среда, Суббота",5-6
1,Минерва Макгоналл,Да,NaN,4-5
2,Альбус Дамблдор,NaN,"Вторник, Пятница, Суббота",5-6


In [7]:
all_slots = ['9.30-10.50', '11.10-12.30', '13.00-14.20', '14.40-16.00', '16.20-17.40', '18.10-19.30', '19.40-21.00']
all_days = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота']
all_subjects = list(groups_data.columns.values)[1:]
all_teachers = list(pd.unique(teachers_data["Преподаватель"]))
all_groups = list(pd.unique(groups_data["Номер группы"]))
print(all_subjects)
print(all_teachers)
print(all_groups)

['Зельеварение, лекция', 'Зельеварение, семинар', 'Защита от темных сил, лекция', 'Защита от темных сил, семинар', 'Трансфигурация, лекция', 'Трансфигурация, семинар']
['Северус Снейп', 'Минерва Макгоналл', 'Альбус Дамблдор']
[1, 2, 3, 4]


In [8]:
def get_subjects_by_teacher(teacher):
  teacher_data = teachers_data[teachers_data["Преподаватель"] == teacher]
  teacher_data = teacher_data.filter(all_subjects)
  available_subjects = list(teacher_data.columns[teacher_data.isin([1]).any()].values)
  subject_inds = [all_subjects.index(subject) for subject in available_subjects]
  return subject_inds

def get_groups_by_subject(subject):
  subject_data = groups_data[groups_data[subject] >= 1]
  available_groups = list(pd.unique(subject_data["Номер группы"]))
  group_inds = [all_groups.index(group) for group in available_groups]
  return group_inds

def get_answers(teacher):
  teacher_data = teachers_data[teachers_data["Преподаватель"] == teacher]
  answers = []
  for day in all_days:
    if pd.isna(teacher_data.iloc[0][day]):
      answers.append(None)
    else:
      slot_inds = [all_slots.index(slot) for slot in teacher_data.iloc[0][day].split(", ")]
      answers.append(slot_inds)
  return answers



def get_subjects_by_group(group):
  group_data = groups_data[groups_data["Номер группы"] == group]
  group_data = group_data.filter(all_subjects)
  needed_subjects = list(group_data.where(group_data >= 1).columns.values)
  subject_inds = [all_subjects.index(subject) for subject in needed_subjects]
  return subject_inds


def get_teachers_by_subject(subject):
  subject_data = teachers_data[teachers_data[subject] == 1]
  available_teachers = list(pd.unique(subject_data["Преподаватель"]))
  teacher_inds = [all_teachers.index(teacher) for teacher in available_teachers]
  return teacher_inds

def get_classes_per_week_for_teacher(teacher):
  teacher_limits_data = limits_data[limits_data["Преподаватель"] == teacher].iloc[0]
  if pd.notna(teacher_limits_data["Количество пар в неделю"]):
    classes_nums = teacher_limits_data["Количество пар в неделю"].split("-")
  return int(classes_nums[0]), int(classes_nums[1])


In [9]:
pref_table = np.zeros((len(groups_data), len(all_subjects), len(teachers_data), len(all_days), len(all_slots)))

for k, teacher in enumerate(all_teachers):
  available_subjects = get_subjects_by_teacher(teacher)
  available_slots = get_answers(teacher)
  for j in available_subjects:
    available_groups = get_groups_by_subject(all_subjects[j])
    for i in available_groups:
      for l, slots_for_day in enumerate(available_slots):
        if slots_for_day is not None:
          for m in slots_for_day:
            pref_table[i, j, k, l, m] = 1


In [10]:
M = 1e5

prob = LpProblem("schedule", LpMaximize)

attends = LpVariable.dicts("x", ((i, j, k, l, m)
                                  for i in range(len(all_groups))
                                  for j in range(len(all_subjects))
                                  for k in range(len(all_teachers))
                                  for l in range(len(all_days))
                                  for m in range(len(all_slots))),
                            cat="Binary")


#Условие 2.1
for i in range(len(all_groups)):
  needed_subjects = get_subjects_by_group(all_groups[i])
  #print(f"Группа {all_groups[i]}")
  for j in needed_subjects:
    #Группа должна посещать указанное количество лекций/семинаров каждого нужного предмета у кого-то из учителей, которые ведут этот предмет
    available_teachers = get_teachers_by_subject(all_subjects[j])
    num = groups_data[groups_data["Номер группы"] == all_groups[i]][all_subjects[j]].iloc[0]
    #print(all_subjects[j], num)
    prob += lpSum([attends[(i, j, k, l, m)]
                  for k in available_teachers
                  for l in range(len(all_days))
                  for m in range(len(all_slots))]) == num
    #Нельзя ходить на лекцию/семинар к учителю, который не ведет этот предмет
    unavailable_teachers = list(set(range(len(all_teachers))) - set(available_teachers))
    prob += lpSum([attends[(i, j, k, l, m)]
                  for k in unavailable_teachers
                  for l in range(len(all_days))
                  for m in range(len(all_slots))]) == 0
  #Нельзя ходить на ненужные предметы
  unneeded_subjects = list(set(range(len(all_subjects))) - set(needed_subjects))
  for j in unneeded_subjects:
    prob += lpSum([attends[(i, j, k, l, m)]
                  for k in range(len(all_teachers))
                  for l in range(len(all_days))
                  for m in range(len(all_slots))]) == 0




#Условие 2.2
#Группа посещает не более одного слота одновременно
for i in range(len(all_groups)):
  for l in range(len(all_days)):
    for m in range(len(all_slots)):
      prob += lpSum([attends[(i, j, k, l, m)]
                     for j in range(len(all_subjects))
                     for k in range(len(all_teachers))]) <= 1


#Условие 3.1
#Преподаватель ведет семинар не более чем у одной группы одновременно, лекцию не более чем у количества групп, которые записаны
for k in range(len(all_teachers)):
  for j in range(len(all_subjects)):
    num_classes = 1;
    if "лекция" in all_subjects[j]:
      num_classes = len(get_groups_by_subject(all_subjects[j]))
    for l in range(len(all_days)):
      for m in range(len(all_slots)):
        prob += lpSum([attends[(i, j, k, l, m)]
                      for i in range(len(all_groups))]) <= num_classes


#Условие 3.2 
#Каждый преподаватель ведет не более одного предмета одновременно

u = LpVariable.dicts("u", ((j, k, l, m)
                    for j in range(len(all_subjects))
                    for k in range(len(all_teachers))
                    for l in range(len(all_days))
                    for m in range(len(all_slots))),
               cat="Binary")



for j in range(len(all_subjects)):
  for k in range(len(all_teachers)):
    for l in range(len(all_days)):
      for m in range(len(all_slots)):
        prob += M * u[(j, k, l, m)] >= lpSum([attends[(i, j, k, l, m)]
                                        for i in range(len(all_groups))])
    

for k in range(len(all_teachers)):
  for l in range(len(all_days)):
      for m in range(len(all_slots)):
          prob += lpSum([u[(j, k, l, m)] for j in range(len(all_subjects))]) <= 1


#Условие 3.6
#Если это лекция, то на ней должны присутствовать все записанные группы одновременно

for j in range(len(all_subjects)):
  if "лекция" in all_subjects[j]:
    groups_num = len(get_groups_by_subject(all_subjects[j]))
    for k in range(len(all_teachers)):
      for l in range(len(all_days)):
        for m in range(len(all_slots)):
          prob += u[(j, k, l, m)] * groups_num == lpSum([attends[(i, j, k, l, m)] for i in range(len(all_groups))])

#Условие 3.7 
#Лекции/семинары должен вести один и тот же преподаватель

q = LpVariable.dicts("q", ((i, j, k)
                    for i in range(len(all_groups))
                    for j in range(len(all_subjects))
                    for k in range(len(all_teachers))),
               cat="Binary")


for i in range(len(all_groups)):
  for j in range(len(all_subjects)):
    for k in range(len(all_teachers)):
      prob += M * q[(i, j, k)] >= lpSum([attends[(i, j, k, l, m)]
                                        for l in range(len(all_days))
                                        for m in range(len(all_slots))])
      
for i in range(len(all_groups)):
  for j in range(len(all_subjects)):
    prob += lpSum([q[(i, j, k)] for k in range(len(all_teachers))])  <= 1



#Условие 3.3
#Отсутствие пар в определенные дни
for k in range(len(all_teachers)):
  forbidden_days_answer = limits_data[limits_data["Преподаватель"] == all_teachers[k]].iloc[0]["Дни без пар"]
  if pd.notna(forbidden_days_answer):
    forbidden_days = [all_days.index(day) for day in forbidden_days_answer.split(", ")]
    for l in forbidden_days:
      prob += lpSum([attends[(i, j, k, l, m)]
                    for i in range(len(all_groups))
                    for j in range(len(all_subjects))
                    for m in range(len(all_slots))]) == 0

#Условие 3.4
#Все пары в один день
y = LpVariable.dicts("y", ((k, l)
                    for k in range(len(all_teachers))
                    for l in range(len(all_days))),
               cat="Binary")

for k in range(len(all_teachers)):
  for l in range(len(all_days)):
    prob += M * y[(k, l)] >= lpSum([attends[(i, j, k, l, m)]
                                    for i in range(len(all_groups))
                                    for j in range(len(all_subjects))
                                    for m in range(len(all_slots))])
    

for k in range(len(all_teachers)):
  in_one_day_answer = limits_data[limits_data["Преподаватель"] == all_teachers[k]]
  if in_one_day_answer.iloc[0]["Все пары в один день"] == "Да": 
    prob += lpSum([y[(k, l)] for l in range(len(all_days))]) == 1

#Условие 3.5
#Ограничение по количеству пар для преподавателей

for k in range(len(all_teachers)):
  mn, mx = get_classes_per_week_for_teacher(all_teachers[k])
  prob += mn <= lpSum([u[(j, k, l, m)]
                 for j in range(len(all_subjects))
                 for l in range(len(all_days))
                 for m in range(len(all_slots))]) <= mx


#Целевая функция -- максимизация довольных преподавателей
tgt1 = LpVariable("tgt1", 
                  cat="Integer")

prob += lpSum([pref_table[i, j, k, l, m] * attends[(i, j, k, l, m)]
               for i in range(len(all_groups))
               for j in range(len(all_subjects))
               for k in range(len(all_teachers)) 
               for l in range(len(all_days)) 
               for m in range(len(all_slots))]) == tgt1
prob += tgt1

In [11]:
prob.solve()

1

In [12]:
print("Значение целевой функции:", prob.objective.value())

Значение целевой функции: 19.0


In [13]:
prob_res = prob.variablesDict()
res = np.chararray((len(all_teachers), len(all_slots), len(all_days)), itemsize=100, unicode=True).astype(str)
res[:] = ""
for k in range(len(all_teachers)):
  print(f"\nПреподаватель:{all_teachers[k]}")
  for l in range(len(all_days)):
    for m in range(len(all_slots)):
      subjects = set()
      for j in range(len(all_subjects)):
        for i in range(len(all_groups)):
          if prob_res[f"x_({i},_{j},_{k},_{l},_{m})"].value() == 1:
            res[k, m, l] += str(all_groups[i]) + ','
            subjects.add(all_subjects[j])
      if (len(res[k, m, l]) > 0):
        res[k, m, l] += str(list(subjects))
        print(f"{all_days[l]} {all_slots[m]}  {res[k, m, l]}")

pd.DataFrame(res[2], columns=all_days, index=all_slots)


Преподаватель:Северус Снейп
Вторник 13.00-14.20  2,['Защита от темных сил, семинар']
Вторник 16.20-17.40  1,2,['Зельеварение, лекция']
Вторник 18.10-19.30  1,2,['Зельеварение, лекция']
Четверг 9.30-10.50  1,['Зельеварение, семинар']
Четверг 11.10-12.30  4,['Защита от темных сил, семинар']
Пятница 13.00-14.20  1,['Зельеварение, семинар']

Преподаватель:Минерва Макгоналл
Понедельник 9.30-10.50  3,['Трансфигурация, семинар']
Понедельник 11.10-12.30  3,['Защита от темных сил, семинар']
Понедельник 13.00-14.20  1,['Защита от темных сил, семинар']
Понедельник 14.40-16.00  1,2,3,4,['Защита от темных сил, лекция']
Понедельник 16.20-17.40  3,['Трансфигурация, семинар']

Преподаватель:Альбус Дамблдор
Понедельник 11.10-12.30  2,['Зельеварение, семинар']
Понедельник 13.00-14.20  3,4,['Трансфигурация, лекция']
Понедельник 18.10-19.30  4,['Трансфигурация, семинар']
Четверг 9.30-10.50  3,4,['Трансфигурация, лекция']
Четверг 11.10-12.30  2,['Зельеварение, семинар']
Четверг 16.20-17.40  4,['Трансфигур

,Понедельник,Вторник,Среда,Четверг,Пятница,Суббота
9.30-10.50,,,,"3,4,['Трансфигурация, лекция']",,
11.10-12.30,"2,['Зельеварение, семинар']",,,"2,['Зельеварение, семинар']",,
13.00-14.20,"3,4,['Трансфигурация, лекция']",,,,,
14.40-16.00,,,,,,
16.20-17.40,,,,"4,['Трансфигурация, семинар']",,
18.10-19.30,"4,['Трансфигурация, семинар']",,,,,
19.40-21.00,,,,,,
